# **1.Import libraby and data**

In [ ]:
import pandas as pd
index = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/machine learning project/player_index.csv")
sta = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/machine learning project/2021-2022 Football Player Stats.csv",
                sep = ';', encoding = 'latin1')
print(index.shape)
print(sta.shape)

(2976, 46)
(2921, 143)


# **2.Remove duplicated records**

## a.2021-2022 Football Player Stats.csv

In [ ]:
player = sta['Player'].unique()
player_boolean = [True]* len(player)
sta_boolean = [0] * len(sta['Player'])
for i in range(len(sta['Player'])):
    for j in range(len(player)):
        if sta['Player'][i] == player[j] and  player_boolean[j] == True:
            player_boolean[j] = False
            sta_boolean[i] = 1

sta['bool'] = sta_boolean
sta = sta[sta['bool'] == 1]

=> we can see that : 136 duplicated records removed in 2021-2022 Football Player Stats.csv

In [ ]:
sta.shape

(2786, 144)

## b.player_index.csv

In [ ]:
index = index.drop_duplicates(subset=["name"], keep=False)
print(index.shape)

(2974, 46)


=> we removed 2 duplicated records 

# **3.Join 2 tables based on player's name in 2 tables**

## a.Idea based on full name

Idea 2 full names taken from 2 tables are the same: if we have at least 2 similar words in any 2 full names of 2 tables, then those 2 full names are the full name of 1 player

In [ ]:
# list of full names are the same in 2 tables
name_sta = []
name_index = []
for i in index['name']:
    for j in sta['Player']:
        name1 = i.strip().split(' ')
        name2 = j.strip().split(' ')
        count = 0
        for a in name1:
            if a in name2:
                count = count + 1
        if (count > 1 ):
            name_sta.append(j)
            name_index.append(i)          

In [ ]:
# we remove full names in 2021-2022 Football Player Stats that don't exist in list of full names above
count_sta =[]
for i in sta['Player']:
    if i in name_sta:
        count_sta.append(name_sta.index(i))
    else:
        count_sta.append(0)
sta['count'] = count_sta
sta = sta[sta['count'] != 0 ]   

In [ ]:
# we remove full names in player_index.csv that don't exist in list of full names above
count_index = []
for i in index['name']:
    if i in name_index:
        count_index.append(name_index.index(i))
    else:
        count_index.append(0)
index['count'] = count_index
index = index[index['count'] != 0 ]


## b. Select attributes of two tables that are related to player values

In [ ]:
data_sta = sta[['Player', 'MP', 'Min', 'Goals', 'Assists', 'CrdY', 'CrdR','count','Comp','Shots',]]

data_index = index.copy()

## c.Combine 2 tables

In [ ]:
final_data = pd.merge(data_sta, data_index, on='count')
final_data.head()

,Player,MP,Min,Goals,Assists,CrdY,CrdR,count,Comp,Shots,...,age,best_position,club,name,overall,position,potential,preferred_foot,value,wage
0,Yunis Abdelhamid,34,2983,0.06,0.00,0.15,0.03,253,Ligue 1,0.54,...,33,CB,Stade de Reims,Yunis Abdelhamid,76,LCB,76,Left,€3.6M,€23K
1,Salis Abdul Samed,31,2462,0.04,0.00,0.44,0.11,1065,Ligue 1,0.66,...,21,CDM,Clermont Foot 63,Salis Abdul Samed,70,SUB,75,Right,€2.2M,€7K
2,Laurent Abergel,34,2956,0.00,0.06,0.27,0.00,25,Ligue 1,0.91,...,28,CDM,FC Lorient,Laurent Abergel,75,CDM,75,Right,€4.9M,€18K
3,Dickson Abiama,24,726,0.00,0.12,0.37,0.00,1937,Bundesliga,2.22,...,22,ST,SpVgg Greuther Fürth,Dickson Abiama,68,SUB,76,Right,€2.7M,€9K
4,Francesco Acerbi,30,2536,0.14,0.00,0.07,0.04,941,Serie A,0.57,...,33,CB,Lazio,Francesco Acerbi,83,LCB,83,Left,€17.5M,€75K


# **4. Check missing values**

In [ ]:
data = final_data.copy()

In [ ]:
# check missing values
data.isnull().sum()[data.isnull().sum() != 0]

Series([], dtype: int64)

=> we don't have any missing values

# **5. Processing with position and best_position**

In [ ]:
# see position of players
data['position'].unique()

array(['LCB', 'SUB', 'CDM', 'LB', 'RES', 'LM', 'LDM', 'RB', 'RCB', 'LS',
       'LCM', 'RCM', 'RM', 'LWB', 'RDM', 'ST', 'CM', 'RW', 'RS', 'GK',
       'CAM', 'RWB', 'RF', 'LF', 'CB', 'LAM', 'CF', 'LW', 'RAM'],
      dtype=object)

In [ ]:
# we replace 'SUB' and 'RES' position by best_positon
for i in range(len(data['position'])):
    if data['position'][i] == 'SUB' or data['position'][i] == 'RES':
        data['position'][i] = data['best_position'][i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
# we make 1 column with only 4 positions in common : goalkeeper, defender, midfielder, striker
position_common = []
for i in data['position']:
    if i in ['GK']:
        position_common.append("goalkeeper")
    elif i in ['LCB', 'LB', 'RB', 'RCB', 'LWB', 'CB','RWB']:
        position_common.append("defender")
    elif i in ['CDM', 'CAM', 'LM', 'CM', 'LDM', 'LCM', 'RM', 'RCM', 'RDM', 'LAM', 'RAM']:
        position_common.append("midfielder")
    elif i in ['ST', 'LS','RW', 'RS', 'RF', 'LW', 'LF', 'CF' ]:
        position_common.append("striker")
    else:
        continue

# add position_column into data tables
data['position common'] = position_common

# **6. Processing with value and wage:**

In [ ]:
market_value = []
for i in range(len(data['value'])):
    if data['value'][i][-1] == 'K':
        market_value.append( float(data['value'][i][1 : -1]) * 1_000)
    else:
        market_value.append( float(data['value'][i][1 : -1]) * 1_000_000)
data['market value'] = market_value
data = data.drop(columns = ['value'])

In [ ]:
wage_value = []
for i in range(len(data['wage'])):
    if data['wage'][i][-1] == 'K':
        wage_value.append( float(data['wage'][i][1 : -1]) * 1_000)
    else:
        wage_value.append(float(data['wage'][i][1:]))
data['wage value'] = wage_value
data = data.drop(columns = ['wage'])

In [ ]:
data.head()

,Player,MP,Min,Goals,Assists,CrdY,CrdR,count,Comp,Shots,...,best_position,club,name,overall,position,potential,preferred_foot,position common,market value,wage value
0,Yunis Abdelhamid,34,2983,0.06,0.00,0.15,0.03,253,Ligue 1,0.54,...,CB,Stade de Reims,Yunis Abdelhamid,76,LCB,76,Left,defender,3600000.0,23000.0
1,Salis Abdul Samed,31,2462,0.04,0.00,0.44,0.11,1065,Ligue 1,0.66,...,CDM,Clermont Foot 63,Salis Abdul Samed,70,CDM,75,Right,midfielder,2200000.0,7000.0
2,Laurent Abergel,34,2956,0.00,0.06,0.27,0.00,25,Ligue 1,0.91,...,CDM,FC Lorient,Laurent Abergel,75,CDM,75,Right,midfielder,4900000.0,18000.0
3,Dickson Abiama,24,726,0.00,0.12,0.37,0.00,1937,Bundesliga,2.22,...,ST,SpVgg Greuther Fürth,Dickson Abiama,68,ST,76,Right,striker,2700000.0,9000.0
4,Francesco Acerbi,30,2536,0.14,0.00,0.07,0.04,941,Serie A,0.57,...,CB,Lazio,Francesco Acerbi,83,LCB,83,Left,defender,17500000.0,75000.0


# **7. Remove redundant columns**

In [ ]:
data_processed = data.drop(columns = ['best_position', 'Player','club','position','count' ])

In [ ]:
data_processed.to_csv("/content/drive/MyDrive/Colab Notebooks/machine learning project/processed data.csv")